<a href="https://colab.research.google.com/github/stephrichter/acceptability-judgements/blob/main/Predicting_Sentence_Probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapted from https://github.com/jhlau/acceptability-prediction-in-context/blob/master/code/compute_model_score.py

In [1]:
# upload items from data
# upload unigram pickle file

In [18]:
#!gunzip /content/xlnet-large-cased-bookcorpus-wikipedia-openwebtext.pickle.gz

gzip: /content/bert-large-uncased-bookcorpus-wikipedia.pickle.gz: No such file or directory


In [3]:
!pip install sentencepiece
!pip install transformers
!pip install datasets

     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 28.5 MB/s 
     |████████████████████████████████| 61 kB 511 kB/s 
     |████████████████████████████████| 596 kB 41.4 MB/s 
     |████████████████████████████████| 3.3 MB 30.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 5.2 MB/s 
     |████████████████████████████████| 132 kB 46.1 MB/s 
     |████████████████████████████████| 243 kB 27.6 MB/s 
     |████████████████████████████████| 1.1 MB 38.4 MB/s 
     |████████████████████████████████| 160 kB 44.7 MB/s 
     |████████████████████████████████| 192 kB 30.5 MB/s 
     |████████████████████████████████| 271 kB 47.8 MB/s 


In [31]:
MODELS = ['bert-large-uncased',
          'xlnet-base-cased']

####################
## argument "parser"
####################

xlnet_pickle = '/content/xlnet-large-cased-bookcorpus-wikipedia-openwebtext.pickle'
bert_pickle = '/content/bert-large-uncased-bookcorpus-wikipedia.pickle'

class Args:
    model_name = MODELS[0]
    item_path = '/content/exp3_items_normalized.tsv'
    unigram_pickle = bert_pickle if model_name.startswith('bert') else xlnet_pickle

    device = 'cuda'
    use_context = False # can't be True
    xlnet_bidir = True

args = Args()

In [32]:
import pandas as pd

#df = pd.read_csv(data_path, skiprows=5)
df = pd.read_csv(args.item_path, sep='\t')

In [23]:
"""
Author:         Jey Han Lau
Date:           Jul 19
"""

import sys
import argparse
import torch
import math
import pickle
import numpy as np
from tqdm import tqdm
#from calc_corr import get_sentence_data
#from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import BertTokenizer, BertForMaskedLM, XLNetTokenizer, XLNetLMHeadModel 
from scipy.stats.mstats import pearsonr as corr
from scipy.special import softmax

#global
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> """

###########
#functions#
###########
def model_score(tokenize_input, tokenize_context, model, tokenizer, device, args):

    if args.model_name.startswith("gpt"):

        if not args.use_context:

            #prepend the sentence with <|endoftext|> token, so that the loss is computed correctly
            tensor_input = torch.tensor([[50256] + tokenizer.convert_tokens_to_ids(tokenize_input)], device=device)
            labels = torch.tensor([[50256] + tokenizer.convert_tokens_to_ids(tokenize_input)], device=device)
            labels[:,:1] = -1
            loss = model(tensor_input, labels=tensor_input)

        else:
            
            tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_context+tokenize_input)], device=device)
            labels = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_context+tokenize_input)], device=device)
            #-1 label for context (loss not computed over these tokens)
            labels[:,:len(tokenize_context)] = -1
            loss = model(tensor_input, labels=labels)

        return float(loss[0]) * -1.0 * len(tokenize_input)

    elif args.model_name.startswith("bert"):

        batched_indexed_tokens = []
        batched_segment_ids = []

        if not args.use_context:
            tokenize_combined = ["[CLS]"] + tokenize_input + ["[SEP]"]
        else:
            tokenize_combined = ["[CLS]"] + tokenize_context + tokenize_input + ["[SEP]"]

        for i in range(len(tokenize_input)):

            # Mask a token that we will try to predict back with `BertForMaskedLM`
            masked_index = i + 1 + (len(tokenize_context) if args.use_context else 0)
            tokenize_masked = tokenize_combined.copy()
            tokenize_masked[masked_index] = '[MASK]'
            #unidir bert
            #for j in range(masked_index, len(tokenize_combined)-1):
            #    tokenize_masked[j] = '[MASK]'

            # Convert token to vocabulary indices
            indexed_tokens = tokenizer.convert_tokens_to_ids(tokenize_masked)
            # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
            segment_ids = [0]*len(tokenize_masked)

            batched_indexed_tokens.append(indexed_tokens)
            batched_segment_ids.append(segment_ids)

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor(batched_indexed_tokens, device=device)
        segment_tensor = torch.tensor(batched_segment_ids, device=device)

        # Predict all tokens
        with torch.no_grad():
            outputs = model(tokens_tensor, token_type_ids=segment_tensor)
            predictions = outputs[0]

        # go through each word and sum their logprobs
        lp = 0.0
        for i in range(len(tokenize_input)):
            masked_index = i + 1 + (len(tokenize_context) if args.use_context else 0)
            predicted_score = predictions[i, masked_index]
            predicted_prob = softmax(predicted_score.cpu().numpy())
            lp += np.log(predicted_prob[tokenizer.convert_tokens_to_ids([tokenize_combined[masked_index]])[0]])

        return lp

    elif args.model_name.startswith("xlnet"):

        tokenize_ptext = tokenizer.tokenize(PADDING_TEXT.lower())

        if not args.use_context:
            tokenize_input2 = tokenize_ptext + tokenize_input
        else:
            tokenize_input2 = tokenize_ptext + tokenize_context + tokenize_input

        # go through each word and sum their logprobs
        lp = 0.0
        for max_word_id in range((len(tokenize_input2)-len(tokenize_input)), (len(tokenize_input2))):

            sent = tokenize_input2[:]
            input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(sent)], device=device)
            perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float, device=device)

            #if not bidir, mask target word + right/future words
            if not args.xlnet_bidir:
                perm_mask[:, :, max_word_id:] = 1.0
            #if bidir, mask only the target word
            else:
                perm_mask[:, :, max_word_id] = 1.0

            target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float, device=device)
            target_mapping[0, 0, max_word_id] = 1.0

            with torch.no_grad():
                outputs = model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)
                next_token_logits = outputs[0]

            word_id = tokenizer.convert_tokens_to_ids([tokenize_input2[max_word_id]])[0]
            predicted_prob = softmax((next_token_logits[0][-1]).cpu().numpy())
            lp += np.log(predicted_prob[word_id])

        return lp

In [24]:
#Load pre-trained model and tokenizer
if args.model_name.startswith("gpt"):
    model = GPT2LMHeadModel.from_pretrained(args.model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(args.model_name)
elif args.model_name.startswith("bert"):
    model = BertForMaskedLM.from_pretrained(args.model_name)
    tokenizer = BertTokenizer.from_pretrained(args.model_name,
        do_lower_case=(True if "uncased" in args.model_name else False))
elif args.model_name.startswith("xlnet"):
    tokenizer = XLNetTokenizer.from_pretrained(args.model_name)
    model = XLNetLMHeadModel.from_pretrained(args.model_name)
else:
    print("Supported models: gpt, bert and xlnet only.")
    raise SystemExit

#put model to device (GPU/CPU)
device = torch.device(args.device)
model.to(device)

#eval mode; no dropout
model.eval()

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [33]:
######
#main#
######
#def main(args):

#sentence and human ratings
#sentencexdata = get_sentence_data(args.input_csv)
#human_ratings = pickle.load(open(args.human_rating_pickle, "rb"))

#unigram frequencies
unigram_freq = pickle.load(open(args.unigram_pickle, "rb"))
unigram_total = sum(unigram_freq.values()) 

#system scores
lps = []
mean_lps = []
pen_lps = []
div_lps = []
sub_lps = []
slors = []
pen_slors = []
sent_ids = []

#loop through each sentence and compute system scores
y = [] #human mean rating
sent_total = 0
for sent_id, sent in tqdm(enumerate(df['sentence'])):

    #y.append(np.mean(ratings))

    #text = sentencexdata[sent_id]["SENTENCE"]
    text = sent
    #uppercase first character
    #text = text[0].upper() + text[1:]
    tokenize_input = tokenizer.tokenize(text)
    text_len = len(tokenize_input)

    if args.use_context:
        context = sentencexdata[sent_id]["CONTEXT"].replace("\t", " ")
        tokenize_context = tokenizer.tokenize(context)
    else:
        tokenize_context = None

    #unigram logprob
    uni_lp = 0.0
    for w in tokenize_input:
        uni_lp += math.log(float(unigram_freq[w])/unigram_total)

    #compute sentence logprob
    lp = model_score(tokenize_input, tokenize_context, model, tokenizer, device, args)

    #acceptability measures
    penalty = ((5+text_len)**0.8 / (5+1)**0.8)
    lps.append(lp)
    mean_lps.append(lp/text_len)
    pen_lps.append( lp / penalty )
    div_lps.append(-lp / uni_lp)
    sub_lps.append(lp - uni_lp)
    slors.append((lp - uni_lp) / text_len)
    pen_slors.append((lp - uni_lp) / penalty)
    sent_ids.append(sent_id)

    sent_total += 1

256it [00:38,  6.70it/s]


In [34]:
for lp, sent in zip(pen_lps, df['sentence']):
  print(lp, sent)

-15.465480137043127 Which detective thinks that Paul stole the necklace?
-17.915050592124047 Which police officer thinks that Matt chased the bus?
-20.76065053923522 Which manager thinks that Tom sold the television?
-17.98247574749852 Which soldier thinks that Stacey wrote the letter?
-16.225811857937067 Which agent thinks that Aaron bought the house?
-16.430669569413958 Which teacher thinks that George read the book?
-15.962689406202736 Which girl thinks that Heather saw the movie?
-14.335212774340636 Which guest thinks that Casey baked the cake?
-12.165226839147728 Which necklace does the detective think that Paul stole?
-19.169093494467457 Which bus does the police officer think that Matt chased?
-16.748709141038745 Which television does the manager think that Tom sold?
-14.364109208720741 Which letter does the soldier think that Stacey wrote?
-10.758135765576757 Which house does the agent think that Aaron bought?
-10.45655492730146 Which book does the teacher think that George rea

In [35]:
df['penalized_logprob'] = pen_lps

In [37]:
df.to_csv('exp3_items_normalized.scored.tsv', sep='\t', index=False, header=True)

In [36]:
df

,item_code,sentence,penalized_logprob
0,WH.whe.non.sh.01,Which detective thinks that Paul stole the nec...,-15.465480
1,WH.whe.non.sh.02,Which police officer thinks that Matt chased t...,-17.915051
2,WH.whe.non.sh.03,Which manager thinks that Tom sold the televis...,-20.760651
3,WH.whe.non.sh.04,Which soldier thinks that Stacey wrote the let...,-17.982476
4,WH.whe.non.sh.05,Which agent thinks that Aaron bought the house?,-16.225812
...,...,...,...
251,WH.sub.isl.lg.04,Which actor do you think the documentary about...,-16.614021
252,WH.sub.isl.lg.05,Which priest do you think the email from criti...,-20.102646
253,WH.sub.isl.lg.06,Which band do you think the picture of accompa...,-10.596768
254,WH.sub.isl.lg.07,Which actress do you think the flyer from prom...,-18.352131
